In [1]:
from pathlib import Path
import sys

ROOT = Path().resolve().parents[1] / "code"
sys.path.append(str(ROOT))
from functions.valid_recall import valid_recall
from functions.read_behaviors import read_train_behaviors,read_dev_behaviors

from gensim.models import Word2Vec
import polars as pl

In [2]:
train = read_train_behaviors()

sentences = (
    train
    .sort("time")
    .group_by("user_id")
    .agg(
        pl.col("history").last()
    )
    .select("history")
    .drop_nulls()
    .to_series()
    .to_list()
)
# 每个 history 本身就是一个“句子”
model = Word2Vec(
    sentences=sentences,
    vector_size=128,
    window=5,
    min_count=5,
    sg=1,          # skip-gram
    workers=8,
    epochs=10
)

model.save("/home/ming/GraduateProject/Data/item2vec.model")
